In [10]:
import numpy as np

# load dialogues_and_speakers.npy
data = np.load('../Book Preprocessing/dialogues_and_speakers.npy')

for dialogue, speaker in data:
    print('Speaker:', speaker)
    print('Dialogue:', dialogue)

# create a list of the first 25 lines of dialogue
all_dialogue_lines = []

# give me only the first 25 lines
for dialogue, speaker in data:
    all_dialogue_lines.append((dialogue, speaker))

Speaker: المعلق
Dialogue: مجنون ليلى الفصل الأول
Speaker: المعلق
Dialogue: (ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)
Speaker: المعلق
Dialogue: ليلى
Speaker: ليلى
Dialogue: دعي الغزْلَ سلمى وحَيِّي معي منارَ الحِجَازِ فَتَى يَثْرِبِ
Speaker: المعلق
Dialogue: (تصافحه سلمى)
Speaker: ليلى
Dialogue:  ويا هِنْدُ هذا أديبُ الحِجازِ هلمِّي بمَقْدَمِهِ رَحِّبِي
Speaker: المعلق
Dialogue: (تصافحه هند ويحتفي به السامرون)
Speaker: المعلق
Dialogue: سعد
Speaker: سعد
Dialogue: أمن يثربٍ أنت آتٍ؟
Speaker: المعلق
Dialogue: ابن ذريح
Speaker: ابن ذريح
Dialogue: أجل من البلدِ القُدُس الطيِّب
Speaker: المعلق
Dialogue: ليلى
Speaker: ليلى
Dialogue: أيابنَ ذَريحٍ لقينا الغمام
Speaker: المعلق
Dialogue: هند
Speaker: هند
Dialogue: وطَافتْ بنا نَفَحَاتُ النبي
Speaker: المعلق
Dialogue: عبلة
Speaker: ال

In [3]:
characters = set()
for dialogue, speaker in all_dialogue_lines:
    characters.add(speaker)

Characters: {'عبلة', 'بشر', 'سعد', 'المعلق', 'ابن ذريح', 'أصوات', 'زياد', 'المهدي', 'منازل', 'سلمى', 'هند', 'قيس', 'فتاة', 'ليلى', 'عفراء'}


In [12]:
# print characters not in character_id dict

character_id = {
    "المعلق": "nPczCjzI2devNBz1zQrb",
    "قيس": "SOYHLrjzK2X1ezoPC6cr",
    "ليلى": "XB0fDUnXU5powFXDhCwa",
    "المهدي": "2EiwWnXFnvU5JabPnv8n",
    "زياد": "VR6AewLTigWG4xSOukaG",
    "سعد": "iP95p4xoKVk53GoZ742B",
    "بشر": "iP95p4xoKVk53GoZ742B",
    "ابن ذريح": "N2lVS1w4EtoT3dr4eOWO",
    "عفراء": "Xb7hH8MSUJpSbSDYk0k2",
    "سلمى": "Xb7hH8MSUJpSbSDYk0k2",
    "هند": "Xb7hH8MSUJpSbSDYk0k2",
    "عبلة": "Xb7hH8MSUJpSbSDYk0k2",
    "فتاة": "Xb7hH8MSUJpSbSDYk0k2",
    "أصوات": "nPczCjzI2devNBz1zQrb",
    "منازل": "iP95p4xoKVk53GoZ742B"
}


for character in characters:
    if character not in character_id:
        print('Character not in character_id:', character)

In [ ]:
import requests
import json
import os

# Load API keys from a JSON file
with open('api_keys.json') as file:
    api_keys = json.load(file)

# Convert the API keys dictionary to a list of keys
api_key_list = list(api_keys.values())

# Function to update the API key in headers
def update_api_key(headers, new_key):
    headers["xi-api-key"] = new_key
    return headers

headers = {
    "xi-api-key": api_key_list[0],  # Start with the first API key
    "Content-Type": "application/json"
}

if not os.path.exists('chapter_1'):
    os.mkdir('chapter_1')

for audioNum, (dialogue, speaker) in enumerate(all_dialogue_lines):
    payload = {
        "model_id": "eleven_multilingual_v1",
        "text": f"{dialogue}"
    }
    
    if speaker not in character_id:
        print(f"Skipping line {audioNum+1} due to missing character ID.")
        continue

    url = f"https://api.elevenlabs.io/v1/text-to-speech/{character_id[speaker]}"

    for api_key in api_key_list:
        response = requests.post(url=url, json=payload, headers=update_api_key(headers, api_key))

        if response.status_code == 200:
            with open(f"chapter_1/{audioNum+1}-{speaker}.mp3", 'wb') as audio_file:
                audio_file.write(response.content)
            print(f"Audio file {audioNum+1} saved successfully.")
            break  # Exit the API key loop on success
        else:
            print(f"Retry with next API key due to status code:", response.status_code)
    else:
        # This part is executed only if the loop was not broken
        print(f"Failed to retrieve audio {audioNum+1}. Tried all API keys.")
        break  # Exit the dialogue loop since all keys failed

In [19]:
# combine all the audio files into one
from moviepy.editor import concatenate_audioclips, AudioClip, AudioFileClip

audio_clips = []

for audioNum, (dialogue, speaker) in enumerate(all_dialogue_lines):
    if speaker in character_id:
        audio_clips.append(AudioFileClip(f"chapter_1/{audioNum+1}-{speaker}.mp3"))
    else:
        print(f"Skipping line {audioNum+1} due to missing character ID.")

# Function to create a silent audio clip of a specified duration
def make_silent_clip(duration):
    # Creating a silent clip. This function generates silence.
    return AudioClip(lambda t: np.zeros(1), duration=duration, fps=44100)

# Duration of the pause in seconds
pause_duration = 0.25

# Create silent clips for pauses
silent_clip = make_silent_clip(pause_duration)

# Insert silent clips between the audio clips
audio_clips_with_pause = [
    clip for pair in zip(
        audio_clips, [silent_clip] * (len(audio_clips) - 1)) for clip in pair] + [audio_clips[-1]]

# Concatenate all clips including the silent ones
final_audio = concatenate_audioclips(audio_clips_with_pause)

final_audio.write_audiofile("chapter1.mp3")
print("Combined audio file saved successfully.")

MoviePy - Writing audio in combined_audio.mp3


MoviePy - Done.
Combined audio file saved successfully.
